### Import Modules

In [24]:
from datetime import datetime
import numpy as np
import pandas as pd
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.stats import shapiro, normaltest
from tpot import TPOTClassifier
from hpsklearn import HyperoptEstimator, pca, min_max_scaler, standard_scaler
from hpsklearn import xgboost_classification, random_forest, ada_boost, gradient_boosting, extra_trees
from hpsklearn import svc, svc_linear, svc_rbf, svc_poly, svc_sigmoid, liblinear_svc
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing
from hyperopt import tpe, hp
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, recall_score, classification_report,confusion_matrix,plot_confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import RFE, SelectFromModel, RFECV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif
from matplotlib import pyplot

import warnings
warnings.filterwarnings('ignore')

### Load and Clean Data

In [2]:
df_train = pd.read_csv('Data/otto_group_Product_train.csv')
#df_test = pd.read_csv('Data/otto_group_Product_test.csv')

In [3]:
df_train.head()

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61878 entries, 0 to 61877
Data columns (total 95 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       61878 non-null  int64 
 1   feat_1   61878 non-null  int64 
 2   feat_2   61878 non-null  int64 
 3   feat_3   61878 non-null  int64 
 4   feat_4   61878 non-null  int64 
 5   feat_5   61878 non-null  int64 
 6   feat_6   61878 non-null  int64 
 7   feat_7   61878 non-null  int64 
 8   feat_8   61878 non-null  int64 
 9   feat_9   61878 non-null  int64 
 10  feat_10  61878 non-null  int64 
 11  feat_11  61878 non-null  int64 
 12  feat_12  61878 non-null  int64 
 13  feat_13  61878 non-null  int64 
 14  feat_14  61878 non-null  int64 
 15  feat_15  61878 non-null  int64 
 16  feat_16  61878 non-null  int64 
 17  feat_17  61878 non-null  int64 
 18  feat_18  61878 non-null  int64 
 19  feat_19  61878 non-null  int64 
 20  feat_20  61878 non-null  int64 
 21  feat_21  61878 non-null  int64 
 22

In [5]:
df_train.shape

(61878, 95)

In [6]:
df_train.isnull().sum()

id         0
feat_1     0
feat_2     0
feat_3     0
feat_4     0
          ..
feat_90    0
feat_91    0
feat_92    0
feat_93    0
target     0
Length: 95, dtype: int64

In [7]:
df_train.isnull().sum().sum()

0

In [8]:
null_columns = {}

all_columns = df_train.isnull().sum().sort_values(ascending=False)
for item in all_columns.index:
    if all_columns[item] > 0:
        null_columns[item] = 100* all_columns[item]/len(df)
        
null_columns

{}

In [9]:
# calculate duplicates
dups = df_train.duplicated()
#print(dups)
# report if there are any duplicates
print(dups.any())

False


In [10]:
# summarize the number of unique values in each column
df_train.nunique().sort_values(ascending=True)

target         9
feat_6         9
feat_5        15
feat_21       15
feat_37       18
           ...  
feat_90       91
feat_74      101
feat_19      105
feat_73      115
id         61878
Length: 95, dtype: int64

In [11]:
df_train["target"].values

array(['Class_1', 'Class_1', 'Class_1', ..., 'Class_9', 'Class_9',
       'Class_9'], dtype=object)

In [12]:
df_train["target"].value_counts()

Class_2    16122
Class_6    14135
Class_8     8464
Class_3     8004
Class_9     4955
Class_7     2839
Class_5     2739
Class_4     2691
Class_1     1929
Name: target, dtype: int64

In [13]:
X = df_train.drop(['id', 'target'],axis=1)
y = df_train['target']

In [14]:
y = LabelEncoder().fit_transform(y)

### Train Test Split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [16]:
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
X_train.shape

(43314, 93)

In [17]:
X_test.shape

(18564, 93)

### Find Best Algorithm

#### Find Best Algorithm with Best Params Using HyperoptEstimator AutoML

In [18]:
preproc = hp.choice('myprepros_name', 
                    [
                        [min_max_scaler('myprepros_name.only_norm')],
                        [standard_scaler('myprepros_name.only_std_scaler')],
                        [pca('myprepros_name.only_pca')],
                        [pca('myprepros_name.pca'), min_max_scaler('myprepros_name.norm')],
                        [min_max_scaler('myprepros_name.first_norm'), standard_scaler('myprepros_name.second_std_scaler')],
                        []
                    ])

In [19]:
clf = hp.choice( 'model_name',
            [ random_forest ('model_name.random_forest'),
            ada_boost ('model_name.ada_boost'),
            gradient_boosting ('model_name.gradient_boosting'),
            xgboost_classification ('model_name.xgboost_classification'),
            extra_trees('model_name.extra_trees'),
            svc_linear('model_name.svc_linear'),
            svc_rbf('model_name.svc_rbf'),
            svc('model_name.svc')]) 

In [20]:
#preprocessing=[min_max_scaler('norm_scaler'), pca('my_pca') ]
#classifier=xgboost_classification('xgboost')
#preprocessing=any_preprocessing('pre')
#preprocessing = preproc
#classifier=any_classifier('cla')
#classifier=clf

#01. 100 times with classifier=any_classifier('cla') & preprocessing=any_preprocessing('pre')

#02. 100 times with classifier=clf & preprocessing = preproc

#03. 100 times with classifier=xgboost_classification('xgboost') & preprocessing = preproc

#04. 100 times with classifier=xgboost_classification('xgboost') & preprocessing = any_preprocessing('pre')

In [17]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
accuracy = 0
best_model = None
#for i in range(100):
    #print (i)
model = HyperoptEstimator(  classifier= any_classifier('cla'), 
                                preprocessing= any_preprocessing('pre'), 
                                algo=tpe.suggest, 
                                max_evals=50, 
                                trial_timeout=5000)
    # perform the search
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
    #if acc > accuracy:
        #accuracy = acc
        #best_model = model
        #print (accuracy)

fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

Job Started at: 2020-11-04 10:56:12.812090
100%|█████████████████████████████████████████████████████████████| 50/50 [03:36<00:00,  4.32s/trial, best loss: 0.1785128140386315]
[20:56:21] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Execution time :  11:02:40.515554
Job Ended at: 2020-11-04 21:58:53.327644


In [18]:
print("Accuracy: %.3f" % acc)

Accuracy: 0.827


In [19]:
#summarize the best model
print("Accuracy: %.3f" % acc)
print(model.best_model())

Accuracy: 0.827
{'learner': XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.9783479287357171, colsample_bynode=1,
              colsample_bytree=0.8174662246984075, gamma=0.6596707384106874,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.025933472296031868, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=3600, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0,
              reg_alpha=0.05907593535724862, reg_lambda=1.0398714478065672,
              scale_pos_weight=1, seed=0, subsample=0.5081679750943192,
              tree_method='exact', validate_parameters=1, verbosity=None), 'preprocs': (StandardScaler(with_mean=False),), 'ex_preprocs': ()}


In [50]:
init_time = datetime.now()
model = XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.50029110018247, colsample_bynode=1,
              colsample_bytree=0.9371080486138555, gamma=0.00014999673883672553,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.15017743323882513, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=4,
              reg_alpha=6.402887715271464e-05, reg_lambda=1.0516334360090362,
              scale_pos_weight=1, seed=4, subsample=0.9995865341563067,
              tree_method='exact', validate_parameters=1, verbosity=None)

model.fit(X_train, y_train,verbose=False,
            early_stopping_rounds=10,
            eval_set=[(X_valid,y_valid)])

prediction = model.predict(X_test)
#print(f"F-measure of XGB: {mean(scores):.3f}")
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))

[21:26:53] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Execution time :  0:14:02.334549


In [53]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.74      0.57      0.64       587
           1       0.77      0.84      0.80      4800
           2       0.62      0.58      0.60      2361
           3       0.78      0.57      0.66       787
           4       0.97      0.98      0.97       845
           5       0.95      0.95      0.95      4290
           6       0.77      0.69      0.72       846
           7       0.90      0.94      0.92      2490
           8       0.88      0.88      0.88      1558

    accuracy                           0.83     18564
   macro avg       0.82      0.78      0.79     18564
weighted avg       0.83      0.83      0.82     18564



#### Find Best Algorithm with Best Params Using TPOT AutoML

In [ ]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")

# define model evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search
#model = TPOTClassifier(generations=5, population_size=50, cv=cv, scoring='accuracy', 
                       #verbosity=2, random_state=1, n_jobs=-1)
tpot = TPOTClassifier(max_time_mins=120, cv=cv, scoring='accuracy', 
                       verbosity=2, random_state=1, n_jobs=-1)
# perform the search
tpot.fit(X_train_scaled, y_train)
# export the best model
tpot.export('tpot_Otto_best_model.py')
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

In [ ]:
tpot.fitted_pipeline_

In [ ]:
tpot.score(X_test_scaled, y_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
# Average CV score on the training set was: 0.7517570704095573
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=MultinomialNB(alpha=100.0, fit_prior=False)),
    DecisionTreeClassifier(criterion="entropy", max_depth=10, min_samples_leaf=12, min_samples_split=7)
)
# Fix random state for all the steps in exported pipeline
set_param_recursive(exported_pipeline.steps, 'random_state', 1)

exported_pipeline.fit(X_train, y_train)
prediction = exported_pipeline.predict(X_test)
print(classification_report(y_test, prediction))
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

#### Find Best Algorithm with Best Params Using PyCaret

In [ ]:
from pycaret.classification import * 
from pycaret.datasets import get_data

##### PyCaret Setup

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
#fold_strategy = 'kfold' Default = 'stratifiedkfold'
clf = setup(data = df_train, target = 'target', session_id=123,
            data_split_stratify = True,
            normalize = True, 
            normalize_method = 'minmax',
            transformation = True, 
            transformation_method = 'quantile',
            pca = True,
            pca_method = 'incremental',
            #feature_selection = True,
            #feature_selection_threshold = 0.5,
            #feature_selection_method = 'boruta',
            remove_outliers = True,
            outliers_threshold = 0.05,
            fix_imbalance = True
           )
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

##### Finding optimal model

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
#top3_models = compare_models(n_select = 3)
best = compare_models()
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
best_model = finalize_model(best)
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

In [ ]:
print (best_model)

##### Tune Model

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
#tuned_model = tune_model(best_model, optimize = 'AUC') # 'Recall'
tuned_model = tune_model(best_model, optimize = 'F1')
print(tuned_model)
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

##### Feature Importance Plot

In [ ]:
plot_model(tuned_model, plot='feature')

In [ ]:
plot_model(tuned_model, 'class_report')

##### Predictions

In [ ]:
predictions = predict_model(tuned_model)

#### Find Best Params With RandomizedSearchCV

In [ ]:
estimators = []
estimators.append(('norm', MinMaxScaler()))
estimators.append(('xgb', XGBClassifier(objective='binary:logistic')))
pipeline = Pipeline(estimators)

In [ ]:
parameters = {
                'xgb__n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
                'xgb__max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
                'xgb__learning_rate': [0.0001, 0.001, 0.01, 0.02, 0.1, 0.2, 0.3], 
                'xgb__subsample' : [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0],
                'xgb__colsample_bytree' : [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0],
                'xgb__colsample_bylevel' : [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0],
                'xgb__gamma': [0],
                'xgb__reg_lambda': [0, 1.0, 3.0, 5.0, 7.0, 10.0, 12.0],
                'xgb__scale_pos_weight': [1, 2, 3, 4, 5, 6]
              }

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
kfold = KFold(n_splits=5, random_state=None, shuffle=True)
xgb_grid = RandomizedSearchCV(pipeline,
                        parameters,
                        cv = kfold,
                        n_jobs = -1,
                        verbose=False)
xgb_grid.fit(X_train, y_train)
grid_predictions = xgb_grid.predict(X_test)
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))

In [ ]:
print(classification_report(y_test, grid_predictions))

In [ ]:
xgb_grid.best_params_

### Define Base Model

In [18]:
# evaluate a model
def evaluate_model(X, y, model):
    # define evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    # evaluate model  'accuracy'  'f1_micro'
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    #print('F-measure: %.3f' % score)
    return scores

In [19]:
model = XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.50029110018247, colsample_bynode=1,
              colsample_bytree=0.9371080486138555, gamma=0.00014999673883672553,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.15017743323882513, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=4,
              reg_alpha=6.402887715271464e-05, reg_lambda=1.0516334360090362,
              scale_pos_weight=1, seed=4, subsample=0.9995865341563067,
              tree_method='exact', validate_parameters=1, verbosity=None)

### Evaluate Base Model

In [27]:
def fit_model (X_train, y_train, X_test, y_test):
    init_time = datetime.now()
    print (f"Job Started at: {init_time}")
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print('Accuracy: %.4f' % (accuracy_score(y_test, predictions)))
    print(classification_report(y_test, predictions))
    fin_time = datetime.now()
    print("Execution time : ", (fin_time-init_time))

In [28]:
fit_model (X_train, y_train, X_test, y_test)

Job Started at: 2020-11-20 10:52:35.977893
[10:52:36] WARNING: /workspace/src/learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Accuracy: 0.8283
              precision    recall  f1-score   support

           0       0.74      0.58      0.65       587
           1       0.77      0.84      0.80      4800
           2       0.62      0.58      0.60      2361
           3       0.78      0.57      0.66       787
           4       0.97      0.98      0.97       845
           5       0.95      0.95      0.95      4290
           6       0.76      0.68      0.72       846
           7       0.90      0.94      0.92      2490
           8       0.88      0.88      0.88      1558

    accuracy                           0.83     18564


### Model interpretation with SHAP

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))

#### Explaining single prediction

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0, :], X.iloc[0, :])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[3, :], X.iloc[3, :])

#### Explaining single feature

In [ ]:
shap.dependence_plot('feat_90', shap_values, X)

#### Explaining the entire dataset

In [ ]:
shap.summary_plot(shap_values, X)

### Normality Test

* NN Rule: Normalize Non-Gaussian

* Standardize Only Gaussian-Like Input Variables
* Normalize Only Non-Gaussian Input Variables

In [34]:
def is_normal(data):
    alpha = 0.05
    stat, p = normaltest(data)
    if p > alpha:
        normalTest = True
    else:
        normalTest = False
    
    stat, p = shapiro(data)
    if p > alpha:
        shapiroTest = True
    else:
        shapiroTest = False
        
    return normalTest and shapiroTest

In [35]:
Gaussian_Like = []
Non_Gaussian = []

for i, name in enumerate (X.columns):
    if is_normal(X[name]):
        Gaussian_Like.append(name)
    else:
        Non_Gaussian.append(name)
        
print (f"Gaussian Like columns: {Gaussian_Like}")
print (f"Non-Gaussian Like columns: {Non_Gaussian}")

Gaussian Like columns: []
Non-Gaussian Like columns: ['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_46', 'feat_47', 'feat_48', 'feat_49', 'feat_50', 'feat_51', 'feat_52', 'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58', 'feat_59', 'feat_60', 'feat_61', 'feat_62', 'feat_63', 'feat_64', 'feat_65', 'feat_66', 'feat_67', 'feat_68', 'feat_69', 'feat_70', 'feat_71', 'feat_72', 'feat_73', 'feat_74', 'feat_75', 'feat_76', 'feat_77', 'feat_78', 'feat_79', 'feat_80', 'feat_81', 'feat_82', 'feat_83', 'feat_84', 'feat_85', 'feat_86', 'feat_87', 'feat_88'

In [32]:
# Standardize Only Gaussian-Like Input Variables
# Normalize Only Non-Gaussian Input Variables

In [ ]:
if len(Gaussian_Like)> 0 and len(Non_Gaussian) > 0:
    print("Normality Function is working")

### Remove correlated features

#### Identify Highly Correlated Features

In [20]:
# Create correlation matrix
corr_matrix = X.corr().abs()
print(); print(corr_matrix)


           feat_1    feat_2    feat_3    feat_4    feat_5    feat_6    feat_7  \
feat_1   1.000000  0.031332  0.027807  0.027529  0.042973  0.043603  0.298952   
feat_2   0.031332  1.000000  0.082573  0.134987  0.020926  0.041343  0.222386   
feat_3   0.027807  0.082573  1.000000  0.583523  0.010880  0.004288  0.001294   
feat_4   0.027529  0.134987  0.583523  1.000000  0.017290  0.014059  0.014490   
feat_5   0.042973  0.020926  0.010880  0.017290  1.000000  0.145355  0.075047   
...           ...       ...       ...       ...       ...       ...       ...   
feat_89  0.096851  0.105527  0.174781  0.183715  0.119951  0.035042  0.063511   
feat_90  0.010310  0.515022  0.015068  0.009454  0.004842  0.054034  0.129578   
feat_91  0.037264  0.026383  0.012417  0.010312  0.012012  0.012465  0.068506   
feat_92  0.054777  0.008219  0.066921  0.087631  0.065331  0.015479  0.032261   
feat_93  0.081783  0.054593  0.006814  0.015746  0.002038  0.008521  0.034912   

           feat_8    feat_

In [21]:
# Select upper triangle of correlation matrix
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
print(); print(upper_tri)


         feat_1    feat_2    feat_3    feat_4    feat_5    feat_6    feat_7  \
feat_1      NaN  0.031332  0.027807  0.027529  0.042973  0.043603  0.298952   
feat_2      NaN       NaN  0.082573  0.134987  0.020926  0.041343  0.222386   
feat_3      NaN       NaN       NaN  0.583523  0.010880  0.004288  0.001294   
feat_4      NaN       NaN       NaN       NaN  0.017290  0.014059  0.014490   
feat_5      NaN       NaN       NaN       NaN       NaN  0.145355  0.075047   
...         ...       ...       ...       ...       ...       ...       ...   
feat_89     NaN       NaN       NaN       NaN       NaN       NaN       NaN   
feat_90     NaN       NaN       NaN       NaN       NaN       NaN       NaN   
feat_91     NaN       NaN       NaN       NaN       NaN       NaN       NaN   
feat_92     NaN       NaN       NaN       NaN       NaN       NaN       NaN   
feat_93     NaN       NaN       NaN       NaN       NaN       NaN       NaN   

           feat_8    feat_9   feat_10  ...   feat_

In [22]:
# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.80)]

In [23]:
print(to_drop)

['feat_45']


In [24]:
len(to_drop)

1

#### Drop Marked Features

In [25]:
# Drop features 
if len(to_drop) > 0:
    X = X.drop(X[to_drop], axis=1)
    print (X.head())

   feat_1  feat_2  feat_3  feat_4  feat_5  feat_6  feat_7  feat_8  feat_9  \
0       1       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       1       0   
2       0       0       0       0       0       0       0       1       0   
3       1       0       0       1       6       1       5       0       0   
4       0       0       0       0       0       0       0       0       0   

   feat_10  ...  feat_84  feat_85  feat_86  feat_87  feat_88  feat_89  \
0        0  ...        0        1        0        0        0        0   
1        0  ...        0        0        0        0        0        0   
2        0  ...        0        0        0        0        0        0   
3        1  ...       22        0        1        2        0        0   
4        0  ...        0        1        0        0        0        0   

   feat_90  feat_91  feat_92  feat_93  
0        0        0        0        0  
1        0        

#### Train Test Split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#### Evaluate Base Model

In [27]:
fit_model (X_train, y_train, X_test, y_test)

Job Started at: 2020-11-13 23:49:47.349071
[23:49:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


              precision    recall  f1-score   support

           0       0.75      0.57      0.65       587
           1       0.76      0.84      0.80      4800
           2       0.62      0.57      0.60      2361
           3       0.78      0.57      0.66       787
           4       0.97      0.98      0.98       845
           5       0.95      0.95      0.95      4290
           6       0.77      0.69      0.73       846
           7       0.90      0.94      0.92      2490
           8       0.88      0.88      0.88      1558

    accuracy         

### Combined Feature Selection Ensembles

In [20]:
# get a voting ensemble of models
def get_ensemble(n_features):
    # define the base models
    models, names = list(), list()
    # anova
    fs = SelectKBest(score_func=f_classif, k=n_features)
    anova = Pipeline([('fs', fs), ('m', model)])
    models.append(('anova', anova))
    names.append('anova')
    # mutual information
    fs = SelectKBest(score_func=mutual_info_classif, k=n_features)
    mutinfo = Pipeline([('fs', fs), ('m', model)])
    models.append(('mutinfo', mutinfo))
    names.append('mutinfo')
    # rfe
    fs = RFE(estimator=RandomForestClassifier(), n_features_to_select=n_features)
    rfe = Pipeline([('fs', fs), ('m', model)])
    models.append(('rfe', rfe))
    names.append('rfe')
    # define the voting ensemble
    ensemble = VotingClassifier(estimators=models, voting='hard')
    names.append('ensemble')
    return names, [anova, mutinfo, rfe, ensemble]

In [21]:
k = 92

In [26]:
# get the ensemble model
names, models = get_ensemble(k)
# evaluate each model
results = list()
for model,name in zip(models,names):
    # define the evaluation method
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate the model on the dataset
    n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    # report performance
    print('>%s: %.3f (%.3f)' % (name, mean(n_scores), std(n_scores)))
    results.append(n_scores)

SyntaxError: invalid syntax (<ipython-input-26-47b38cf37d8c>, line 11)

In [25]:
predictions = pipeline.predict(X_test)
print('> %.4f' % (accuracy_score(y_test, predictions)))
print(classification_report(y_test, predictions))

> 0.8281
              precision    recall  f1-score   support

           0       0.73      0.58      0.65       587
           1       0.77      0.84      0.80      4800
           2       0.62      0.58      0.60      2361
           3       0.78      0.57      0.66       787
           4       0.97      0.98      0.97       845
           5       0.95      0.95      0.95      4290
           6       0.77      0.68      0.72       846
           7       0.90      0.94      0.92      2490
           8       0.88      0.88      0.88      1558

    accuracy                           0.83     18564
   macro avg       0.82      0.78      0.79     18564
weighted avg       0.83      0.83      0.83     18564



In [43]:
X_train.isnull().sum().sum()

0

### Convert or export the model into ONNX format

In [49]:
import onnxruntime as rt
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes)
from onnxmltools.convert.xgboost.operator_converters.XGBoost import (
    convert_xgboost)
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster

In [50]:
#For XGBoost only
update_registered_converter(
    XGBClassifier, 'XGBoostXGBClassifier',
    calculate_linear_classifier_output_shapes, convert_xgboost,
    options={'nocl': [True, False], 'zipmap': [True, False]})

In [51]:
model_onnx = convert_sklearn(
    pipeline, 'xgboost_model',
    [('input', FloatTensorType([None, 2]))],
    target_opset=12)

MissingShapeCalculator: Unable to find a shape calculator for type '<class 'sklearn.preprocessing._data.QuantileTransformer'>'.
It usually means the pipeline being converted contains a
transformer or a predictor with no corresponding converter
implemented in sklearn-onnx. If the converted is implemented
in another library, you need to register
the converted so that it can be used by sklearn-onnx (function
update_registered_converter). If the model is not yet covered
by sklearn-onnx, you may raise an issue to
https://github.com/onnx/sklearn-onnx/issues
to get the converter implemented or even contribute to the
project. If the model is a custom model, a new converter must
be implemented. Examples can be found in the gallery.


In [33]:
# And save.
with open("pipeline_xgboost.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

In [ ]:
print("predict", pipeline.predict(X_test[:5]))
print("predict_proba", pipeline.predict_proba(X_test[:1]))

### Load and run the model using ONNX Runtime

In [ ]:
sess = rt.InferenceSession("pipeline_xgboost.onnx")
pred_onx = sess.run(None, {"input": X_test[:5].astype(numpy.float32)})
print("predict", pred_onx[0])
print("predict_proba", pred_onx[1][:1])